In [1]:
import sys
import os
import numpy as np
from copy import deepcopy

sys.path.insert(0, os.path.abspath('./src/'))

import pdb_numpy
from pdb_numpy import Coor, Model, format, abinitio, alignement
from pdb_numpy.format import mmcif, pdb

In [2]:
seq1 = "SGYWITCCPTCDVDINTWVPFYSTELNKPAMIYCSHGDGHWVHAQCMDLAERTLIHLSAGSNKYYCNEHVEIARA"
seq2 = "LPVEKIIREAKKILDELLKRGLIDPELARIAREVLERARKLGNEEAARFVLELIERLRRELS"

ali = alignement.align_seq(seq1, seq2)
print(ali)

alignement.print_align_seq(ali[0], ali[1])


('SGYWITCCPTCDVDINTWVPFYSTELNKPAMIYCSHGDGHWVHAQCMDLAERTLI--HLSAGSNK--YY-CN--EHV--EIARA', '------------------LPVEKI-IREAKKILDELLKRGLIDPELARIA-REVLERARKLGNEEAARFVLELIERLRRELS--')
SGYWITCCPTCDVDINTWVPFYSTELNKPAMIYCSHGDGHWVHAQCMDLAERTLI--HLSAGSNK--YY-CN--EHV--E
                  |*  |  |||   *  |      |  |   |* * ||    | *|||   |  |  *||  *
------------------LPVEKI-IREAKKILDELLKRGLIDPELARIA-REVLERARKLGNEEAARFVLELIERLRRE


IARA
||  
LS--


Identity seq1: 9.33%
Identity seq2: 11.29%
Similarity seq1: 37.33%
Similarity seq2: 45.16%


In [3]:
seq1 = "SGYWITCCPTCDVDINTWVPFYSTELNKPAMIYCSHGDGHWVHAQCMDLAERTLIHLSAGSNKYYCNEHVEIARA"
seq2 = "LPVEKIIREAKKILDELLKRGLIDPELARIAREVLERARKLGNEEAARFVLELIERLRRELS"

ali = alignement.align_seq_cython(seq1, seq2)
print(ali)
alignement.print_align_seq(ali[0], ali[1])

('SGYWITCCPTCDVDINTWVPFYSTELNKPAMIYCSHGDGHWVHAQCMDLAERTL-I-HLSAGSNK--YYCNEHVEI-AR--A', '------------------LPVEKI-IREAKKILDELLKRGLIDPELARIAREVLE-RARKLGNEEAARFVLELIERLRRELS')
SGYWITCCPTCDVDINTWVPFYSTELNKPAMIYCSHGDGHWVHAQCMDLAERTL-I-HLSAGSNK--YYCNEHVEI-AR-
                  |*  |  |||   *  |      |  |   |*|||*     | *|||   |  * |*   * 
------------------LPVEKI-IREAKKILDELLKRGLIDPELARIAREVLE-RARKLGNEEAARFVLELIERLRRE


-A
 |
LS


Identity seq1: 10.67%
Identity seq2: 12.90%
Similarity seq1: 36.00%
Similarity seq2: 43.55%


In [2]:
pdb_numpy.__file__

'/home/murail/Documents/Code/pdb_numpy/src/pdb_numpy/__init__.py'

In [3]:
corr = mmcif.fetch_BioAssembly('8OP8')

In [4]:
rm tmp_8OP8.cif

In [5]:
corr.write('tmp_8OP8.cif')

In [6]:
tmp = Coor('tmp_8OP8.cif')

In [4]:
corr.get_aa_seq()

{'A': 'AAQTNAPWGLARISSTSPGTSTYYYDESAGQGSCVYVIDTGIEASHPEF------VKTYYYSSRDGNGHGTHCAGTVGSRTYGVAKKTQLFGVKVLDDNGSGQYS-------------------KGVVASLSLGGGYSSSVNSAAARLQSSGVMVAVAAGN-----RNYSPASEPS-CTVGAS----------N--S-L-I-------L------------------------------------------------------------------N-QA',
 'B': 'AAASVK'}

In [5]:
corr_pdb = pdb.fetch_BioAssembly('8SOU')

In [6]:
corr_pdb.get_aa_seq()

{'A': 'AAQTNAPWGLARISSTSPGTSTYYYDESAGQGSCVYVIDTGIEASHPEF------VKTYYYSSRDGNGHGTHCAGTVGSRTYGVAKKTQLFGVKVLDDNGSGQYS-------------------KGVVASLSLGGGYSSSVNSAAARLQSSGVMVAVAAGN-----RNYSPASEPS-CTVGAS----------N--S-L-I-------L------------------------------------------------------------------N-QA',
 'B': 'AAASVK'}

In [10]:
CHAIN_LIST = [chr(i) for i in list(range(65, 91)) + list(range(48, 58)) + list(range(97, 123)) + list(range(192, 500))]
print(CHAIN_LIST)

chain = "D-"

count = 0
base = 65
for i, letter in enumerate(chain):
    if i > 0:
        base = 64
    count += 26**i * (ord(letter) - base)
print(count)
if 0 < count < len(CHAIN_LIST):
    print(CHAIN_LIST[count])
else:
    print("0")


['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'À', 'Á', 'Â', 'Ã', 'Ä', 'Å', 'Æ', 'Ç', 'È', 'É', 'Ê', 'Ë', 'Ì', 'Í', 'Î', 'Ï', 'Ð', 'Ñ', 'Ò', 'Ó', 'Ô', 'Õ', 'Ö', '×', 'Ø', 'Ù', 'Ú', 'Û', 'Ü', 'Ý', 'Þ', 'ß', 'à', 'á', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ö', '÷', 'ø', 'ù', 'ú', 'û', 'ü', 'ý', 'þ', 'ÿ', 'Ā', 'ā', 'Ă', 'ă', 'Ą', 'ą', 'Ć', 'ć', 'Ĉ', 'ĉ', 'Ċ', 'ċ', 'Č', 'č', 'Ď', 'ď', 'Đ', 'đ', 'Ē', 'ē', 'Ĕ', 'ĕ', 'Ė', 'ė', 'Ę', 'ę', 'Ě', 'ě', 'Ĝ', 'ĝ', 'Ğ', 'ğ', 'Ġ', 'ġ', 'Ģ', 'ģ', 'Ĥ', 'ĥ', 'Ħ', 'ħ', 'Ĩ', 'ĩ', 'Ī', 'ī', 'Ĭ', 'ĭ', 'Į', 'į', 'İ', 'ı', 'Ĳ', 'ĳ', 'Ĵ', 'ĵ', 'Ķ', 'ķ', 'ĸ', 'Ĺ', 'ĺ', 'Ļ', 'ļ', 'Ľ', 'ľ', 'Ŀ', 'ŀ', 'Ł', 'ł', 'Ń', 'ń', 'Ņ', 'ņ', 'Ň', 'ň', 'ŉ',

In [11]:
list_mmcif = ['8C1L',
'8TEC',
'8AQB',
'7RZJ',
'8S6A',
'7TO8',
'8BQ3',
'7TOA',
'8TMU',
'8KDX',]

coor_list = []

for mmcif_file in list_mmcif:
    print(mmcif_file)
    my_coor = mmcif.fetch_BioAssembly(mmcif_file)
    #my_coor = pdb.fetch_BioAssembly(mmcif_file)
    my_coor.write('tmp_' + mmcif_file + '.pdb', overwrite=True)
    coor_list.append(my_coor)
    #seq = my_coor.get_aa_seq()
    #print(mmcif_file, seq)

8C1L


No cell information in mmCIF file


8TEC


No cell information in mmCIF file


8AQB


...

No cell information in mmCIF file


8TMU


No cell information in mmCIF file


8KDX


No cell information in mmCIF file


In [12]:
my_coor .chain

array(['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',
       'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',
       'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',
       'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',
       'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',
       'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',
       'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',
       'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',
       'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',
       'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',
       'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',
       'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',
       'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',
       'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A

In [13]:
coor_list[0].models[1].len

IndexError: list index out of range

In [ ]:
my_coor = pdb.fetch_BioAssembly('8C1L')
my_coor.models[0].atom_dict['name_resname_elem']

array([['N', 'ALA', 'N'],
       ['CA', 'ALA', 'C'],
       ['C', 'ALA', 'C'],
       ...,
       ['O', 'HOH', 'O'],
       ['O', 'HOH', 'O'],
       ['O', 'HOH', 'O']], dtype='<U4')

In [ ]:
my_coor.select_atoms('name CA')

IndexError: index 57 is out of bounds for axis 0 with size 52

In [ ]:
len(my_coor.models)

2

In [ ]:
test = Coor('../BeEM/example_input/3j6b.cif')
test.write("tmp.pdb", overwrite=True)

In [ ]:
from importlib import reload
abinitio = reload(abinitio)

In [ ]:
file = '../BeEM/example_input/3j6b.cif'

In [ ]:
%%timeit
test = Coor(file)

2.42 s ± 14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit

count = 0
with open(file, 'r') as f:
    for line in f:
        count += 1

31.1 ms ± 287 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
test.write("tmp.pdb", overwrite=True)

4.08 s ± 43.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f format.pdb.get_pdb_string test.write("tmp.pdb", overwrite=True)

Timer unit: 1e-09 s

Total time: 5.20449 s
File: /home/murail/Documents/Code/pdb_numpy/src/pdb_numpy/format/pdb.py
Function: get_pdb_string at line 323

Line #      Hits         Time  Per Hit   % Time  Line Contents
   323                                           def get_pdb_string(pdb_coor):
   324                                               """Return a coor object as a pdb string.
   325                                           
   326                                               Parameters
   327                                               ----------
   328                                               pdb_coor : Coor
   329                                                   Coor object
   330                                               
   331                                               Returns
   332                                               -------
   333                                               str
   334                                                   Coor 

In [ ]:
%lprun -f format.mmcif._parse_raw_mmcif_lines Coor(file)

Timer unit: 1e-09 s

Total time: 4.57734 s
File: /home/murail/Documents/Code/pdb_numpy/src/pdb_numpy/format/mmcif.py
Function: _parse_raw_mmcif_lines at line 461

Line #      Hits         Time  Per Hit   % Time  Line Contents
   461                                           def _parse_raw_mmcif_lines(mmcif_lines):
   462                                               """Parse the mmcif lines and return atom information as a dictionary
   463                                           
   464                                               Parameters
   465                                               ----------
   466                                               mmcif_lines : list
   467                                                   list of pdb lines
   468                                           
   469                                               Returns
   470                                               -------
   471                                               dict
   472 

In [ ]:
line = "ATOM   987    H  H62    . A   A  1  38   ? 345.584 231.000 189.453 1.00   74.87  ? 38   A   A H62    1 "

In [ ]:
import shlex

In [ ]:
%timeit shlex.split(line, posix=False)

69.2 µs ± 1.44 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
%timeit line.split()

575 ns ± 4.87 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [ ]:
import re

In [ ]:
%timeit re.split('\s', line)

9.02 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [ ]:
import subprocess
subprocess.run(["../BeEM/BeEM", "../BeEM/example_input/3j6b.cif"])

3j6b-pdb-bundle1.pdb
3j6b-pdb-bundle2.pdb
3j6b-chain-id-mapping.txt


CompletedProcess(args=['../BeEM/BeEM', '../BeEM/example_input/3j6b.cif'], returncode=0)

In [ ]:
%%timeit
subprocess.run(["../BeEM/BeEM", "../BeEM/example_input/3j6b.cif"])

3j6b-pdb-bundle1.pdb
3j6b-pdb-bundle2.pdb
3j6b-chain-id-mapping.txt
3j6b-pdb-bundle1.pdb
3j6b-pdb-bundle2.pdb
3j6b-chain-id-mapping.txt
3j6b-pdb-bundle1.pdb
3j6b-pdb-bundle2.pdb
3j6b-chain-id-mapping.txt
3j6b-pdb-bundle1.pdb
3j6b-pdb-bundle2.pdb
3j6b-chain-id-mapping.txt
3j6b-pdb-bundle1.pdb
3j6b-pdb-bundle2.pdb
3j6b-chain-id-mapping.txt
3j6b-pdb-bundle1.pdb
3j6b-pdb-bundle2.pdb
3j6b-chain-id-mapping.txt
3j6b-pdb-bundle1.pdb
3j6b-pdb-bundle2.pdb
3j6b-chain-id-mapping.txt
3j6b-pdb-bundle1.pdb
3j6b-pdb-bundle2.pdb
3j6b-chain-id-mapping.txt
755 ms ± 12.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
print(test.data_mmCIF)

{'title': {'title': 'untitled'}, '_entry': {'id': 'XXXX'}, '_cell': {'length_a': '20.63', 'length_b': '4.7', 'length_c': '21.009', 'angle_alpha': '90.0', 'angle_beta': '92.28', 'angle_gamma': '90.0', 'Z_PDB': '2'}, '_atom_site': None}


In [ ]:
test = Coor('/home/murail/Downloads/3ftk.cif')
print(test.crystal_pack)

CRYST1   20.630    4.700   21.009  90.00  92.28  90.00 P 1 21 1    2



In [ ]:
test.write("test.cif", overwrite=True)

In [ ]:
print(test.transformation)

{1: {'chains': ['A', 'B'], 'matrix': [[1.0, 1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 1.0, 0.0], [2.0, 1.0, 0.0, 0.0, 0.0], [2.0, 0.0, 1.0, 0.0, 4.7], [2.0, 0.0, 0.0, 1.0, 0.0], [3.0, 1.0, 0.0, 0.0, 0.0], [3.0, 0.0, 1.0, 0.0, -4.7], [3.0, 0.0, 0.0, 1.0, 0.0]]}, 2: {'chains': ['A', 'B'], 'matrix': [[4.0, -1.0, 0.0, 0.0, 0.0], [4.0, 0.0, 1.0, 0.0, 2.35], [4.0, 0.0, 0.0, -1.0, 0.0], [5.0, -1.0, 0.0, 0.0, 0.0], [5.0, 0.0, 1.0, 0.0, 7.05], [5.0, 0.0, 0.0, -1.0, 0.0], [6.0, -1.0, 0.0, 0.0, 0.0], [6.0, 0.0, 1.0, 0.0, -2.35], [6.0, 0.0, 0.0, -1.0, 0.0]]}, 3: {'chains': ['A', 'B'], 'matrix': [[1.0, 1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 1.0, 0.0]]}}


In [ ]:
test.apply_transformation([1,2,3])
test.write('tmp_trans3.pdb')

In [ ]:
def pdb_parse_transformation(text):
    """Parse the `REMARK 350   BIOMT` information from a pdb file.

    Parameters
    ----------
    text : str
        pdb file

    Returns
    -------
    symetry_dict : dict
        symetry information
    """

    transformation_dict = {}

    for line in text.split("\n"):
        if line[11:23] == "BIOMOLECULE:":
            biomol = int(line[24:])
            transformation_dict[biomol] = {"chains": [], "matrix": []}
        elif line[34:41] == "CHAINS:":
            transformation_dict[biomol]["chains"] += line[42:].split()
        elif line.startswith("REMARK 350   BIOMT"):
            transformation_dict[biomol]["matrix"] += [
                [float(x) for x in line[19:].split()]
            ]

    return transformation_dict

txt = """REMARK 350                                                                      
REMARK 350 COORDINATES FOR A COMPLETE MULTIMER REPRESENTING THE KNOWN           
REMARK 350 BIOLOGICALLY SIGNIFICANT OLIGOMERIZATION STATE OF THE                
REMARK 350 MOLECULE CAN BE GENERATED BY APPLYING BIOMT TRANSFORMATIONS          
REMARK 350 GIVEN BELOW.  BOTH NON-CRYSTALLOGRAPHIC AND                          
REMARK 350 CRYSTALLOGRAPHIC OPERATIONS ARE GIVEN.                               
REMARK 350                                                                      
REMARK 350 BIOMOLECULE: 1                                                       
REMARK 350 AUTHOR DETERMINED BIOLOGICAL UNIT: DIMERIC                           
REMARK 350 APPLY THE FOLLOWING TO CHAINS: A                                     
REMARK 350   BIOMT1   1  1.000000  0.000000  0.000000        0.00000            
REMARK 350   BIOMT2   1  0.000000  1.000000  0.000000        0.00000            
REMARK 350   BIOMT3   1  0.000000  0.000000  1.000000        0.00000            
REMARK 350   BIOMT1   2  1.000000  0.000000  0.000000        0.94015            
REMARK 350   BIOMT2   2  0.000000  1.000000  0.000000        4.78756            
REMARK 350   BIOMT3   2  0.000000  0.000000  1.000000        0.00000"""

txt2 = """REMARK 350                                                                      
REMARK 350 COORDINATES FOR A COMPLETE MULTIMER REPRESENTING THE KNOWN           
REMARK 350 BIOLOGICALLY SIGNIFICANT OLIGOMERIZATION STATE OF THE                
REMARK 350 MOLECULE CAN BE GENERATED BY APPLYING BIOMT TRANSFORMATIONS          
REMARK 350 GIVEN BELOW.  BOTH NON-CRYSTALLOGRAPHIC AND                          
REMARK 350 CRYSTALLOGRAPHIC OPERATIONS ARE GIVEN.                               
REMARK 350                                                                      
REMARK 350 BIOMOLECULE: 1                                                       
REMARK 350 AUTHOR DETERMINED BIOLOGICAL UNIT: TRIMERIC                          
REMARK 350 APPLY THE FOLLOWING TO CHAINS: A                                     
REMARK 350   BIOMT1   1  1.000000  0.000000  0.000000        0.00000            
REMARK 350   BIOMT2   1  0.000000  1.000000  0.000000        0.00000            
REMARK 350   BIOMT3   1  0.000000  0.000000  1.000000        0.00000            
REMARK 350   BIOMT1   2  1.000000  0.000000  0.000000        0.00000            
REMARK 350   BIOMT2   2  0.000000  1.000000  0.000000        4.70000            
REMARK 350   BIOMT3   2  0.000000  0.000000  1.000000        0.00000            
REMARK 350   BIOMT1   3  1.000000  0.000000  0.000000        0.00000            
REMARK 350   BIOMT2   3  0.000000  1.000000  0.000000       -4.70000            
REMARK 350   BIOMT3   3  0.000000  0.000000  1.000000        0.00000            
REMARK 350                                                                      
REMARK 350 BIOMOLECULE: 2                                                       
REMARK 350 AUTHOR DETERMINED BIOLOGICAL UNIT: TRIMERIC                          
REMARK 350 APPLY THE FOLLOWING TO CHAINS: A                                     
REMARK 350   BIOMT1   1 -1.000000  0.000000  0.000000        0.00000            
REMARK 350   BIOMT2   1  0.000000  1.000000  0.000000        2.35000            
REMARK 350   BIOMT3   1  0.000000  0.000000 -1.000000        0.00000            
REMARK 350   BIOMT1   2 -1.000000  0.000000  0.000000        0.00000            
REMARK 350   BIOMT2   2  0.000000  1.000000  0.000000        7.05000            
REMARK 350   BIOMT3   2  0.000000  0.000000 -1.000000        0.00000            
REMARK 350   BIOMT1   3 -1.000000  0.000000  0.000000        0.00000            
REMARK 350   BIOMT2   3  0.000000  1.000000  0.000000       -2.35000            
REMARK 350   BIOMT3   3  0.000000  0.000000 -1.000000        0.00000            
REMARK 350                                                                      
REMARK 350 BIOMOLECULE: 3                                                       
REMARK 350 SOFTWARE DETERMINED QUATERNARY STRUCTURE: MONOMERIC                  
REMARK 350 SOFTWARE USED: PISA                                                  
REMARK 350 APPLY THE FOLLOWING TO CHAINS: A                                     
REMARK 350   BIOMT1   1  1.000000  0.000000  0.000000        0.00000            
REMARK 350   BIOMT2   1  0.000000  1.000000  0.000000        0.00000            
REMARK 350   BIOMT3   1  0.000000  0.000000  1.000000        0.00000"""


#print(pdb_parse_transformation(txt))
pdb_parse_transformation(txt2)
#print()
#print(pdb_parse_transformation(txt2))

{1: {'chains': ['A'],
  'matrix': [[1.0, 1.0, 0.0, 0.0, 0.0],
   [1.0, 0.0, 1.0, 0.0, 0.0],
   [1.0, 0.0, 0.0, 1.0, 0.0],
   [2.0, 1.0, 0.0, 0.0, 0.0],
   [2.0, 0.0, 1.0, 0.0, 4.7],
   [2.0, 0.0, 0.0, 1.0, 0.0],
   [3.0, 1.0, 0.0, 0.0, 0.0],
   [3.0, 0.0, 1.0, 0.0, -4.7],
   [3.0, 0.0, 0.0, 1.0, 0.0]]},
 2: {'chains': ['A'],
  'matrix': [[1.0, -1.0, 0.0, 0.0, 0.0],
   [1.0, 0.0, 1.0, 0.0, 2.35],
   [1.0, 0.0, 0.0, -1.0, 0.0],
   [2.0, -1.0, 0.0, 0.0, 0.0],
   [2.0, 0.0, 1.0, 0.0, 7.05],
   [2.0, 0.0, 0.0, -1.0, 0.0],
   [3.0, -1.0, 0.0, 0.0, 0.0],
   [3.0, 0.0, 1.0, 0.0, -2.35],
   [3.0, 0.0, 0.0, -1.0, 0.0]]},
 3: {'chains': ['A'],
  'matrix': [[1.0, 1.0, 0.0, 0.0, 0.0],
   [1.0, 0.0, 1.0, 0.0, 0.0],
   [1.0, 0.0, 0.0, 1.0, 0.0]]}}

In [ ]:

def parse_transformation(data_mmCIF):
    """Parse the `_pdbx_struct_oper_list` information from a mmcif file.

    Parameters
    ----------
    pdbx_struct_oper_list : dict
        mmcif file information

    Returns
    -------
    symetry_dict : dict
        symetry information
    """
    
    matrix_indexes = [
        ['id', 'matrix[1][1]', 'matrix[1][2]', 'matrix[1][3]', 'vector[1]'],
        ['id', 'matrix[2][1]', 'matrix[2][2]', 'matrix[2][3]', 'vector[2]'],
        ['id', 'matrix[3][1]', 'matrix[3][2]', 'matrix[3][3]', 'vector[3]']
    ]
    transformation_dict = {}
    print('_pdbx_struct_assembly_gen', data_mmCIF['_pdbx_struct_assembly_gen'])
    print('_pdbx_struct_oper_list', data_mmCIF['_pdbx_struct_oper_list'])


    # Extract transformation list:
    # Here with only one transformation
    if 'asym_id_list' in data_mmCIF['_pdbx_struct_assembly_gen']:
        trans_num = 1
        chain_list = [
            chain.strip() for chain in data_mmCIF['_pdbx_struct_assembly_gen']['asym_id_list'].split(',')
        ]
        
        transformation_dict[1] = {"chains": chain_list, "matrix": []}

        for i in range(len(data_mmCIF['_pdbx_struct_oper_list']['value'][0])):
            
            for matrix_index in matrix_indexes:
                local_matrix = []
                for index in matrix_index:
                    local_index = data_mmCIF['_pdbx_struct_oper_list']['col_names'].index(index)
                    local_matrix.append(float(data_mmCIF['_pdbx_struct_oper_list']['value'][local_index][i]))
                transformation_dict[1]["matrix"].append(local_matrix)
        
    # Here with multiple transformation
    else:
        trans_num = len(data_mmCIF['_pdbx_struct_assembly']['value'][0])
        assert trans_num == len(data_mmCIF['_pdbx_struct_assembly_gen']['value'][0])
        
        chain_index = data_mmCIF['_pdbx_struct_assembly_gen']['col_names'].index('asym_id_list')
        local_matrix_index = data_mmCIF['_pdbx_struct_assembly_gen']['col_names'].index('oper_expression')
        
        for i in range(trans_num):
            # Extract chain list and matrix indexes
            chain_list = [
                chain.strip() for chain in data_mmCIF['_pdbx_struct_assembly_gen']['value'][chain_index][i].split(',')
            ]
            matrix_index_list = [
                chain.strip() for chain in data_mmCIF['_pdbx_struct_assembly_gen']['value'][local_matrix_index][i].split(',')
            ]
            transformation_dict[i+1] = {"chains": chain_list, "matrix": []}

            # Extract matrix value
            for j in range(len(data_mmCIF['_pdbx_struct_oper_list']['value'][0])):
                matrix_id = data_mmCIF['_pdbx_struct_oper_list']['value'][0][j]
                
                if matrix_id in matrix_index_list:
                    for matrix_index in matrix_indexes:
                        local_matrix = []
                        for index in matrix_index:
                            local_index = data_mmCIF['_pdbx_struct_oper_list']['col_names'].index(index)
                            local_matrix.append(float(data_mmCIF['_pdbx_struct_oper_list']['value'][local_index][j]))
                        transformation_dict[i+1]["matrix"].append(local_matrix)
        
    return transformation_dict


In [ ]:
parse_transformation(test2.data_mmCIF)

_pdbx_struct_assembly_gen {'col_names': ['assembly_id', 'oper_expression', 'asym_id_list'], 'value': [['1', '2', '3'], ['1,2,3', '4,5,6', '1'], ['A,B', 'A,B', 'A,B']]}
_pdbx_struct_oper_list {'col_names': ['id', 'type', 'name', 'symmetry_operation', 'matrix[1][1]', 'matrix[1][2]', 'matrix[1][3]', 'vector[1]', 'matrix[2][1]', 'matrix[2][2]', 'matrix[2][3]', 'vector[2]', 'matrix[3][1]', 'matrix[3][2]', 'matrix[3][3]', 'vector[3]'], 'value': [['1', '2', '3', '4', '5', '6'], ["'identity operation'", "'crystal symmetry operation'", "'crystal symmetry operation'", "'crystal symmetry operation'", "'crystal symmetry operation'", "'crystal symmetry operation'"], ['1_555', '1_565', '1_545', '2_555', '2_565', '2_545'], ['x,y,z', 'x,y+1,z', 'x,y-1,z', '-x,y+1/2,-z', '-x,y+3/2,-z', '-x,y-1/2,-z'], ['1.0000000000', '1.0000000000', '1.0000000000', '-1.0000000000', '-1.0000000000', '-1.0000000000'], ['0.0000000000', '0.0000000000', '0.0000000000', '0.0000000000', '0.0000000000', '0.0000000000'], ['0.0

{1: {'chains': ['A', 'B'],
  'matrix': [[1.0, 1.0, 0.0, 0.0, 0.0],
   [1.0, 0.0, 1.0, 0.0, 0.0],
   [1.0, 0.0, 0.0, 1.0, 0.0],
   [2.0, 1.0, 0.0, 0.0, 0.0],
   [2.0, 0.0, 1.0, 0.0, 4.7],
   [2.0, 0.0, 0.0, 1.0, 0.0],
   [3.0, 1.0, 0.0, 0.0, 0.0],
   [3.0, 0.0, 1.0, 0.0, -4.7],
   [3.0, 0.0, 0.0, 1.0, 0.0]]},
 2: {'chains': ['A', 'B'],
  'matrix': [[4.0, -1.0, 0.0, 0.0, 0.0],
   [4.0, 0.0, 1.0, 0.0, 2.35],
   [4.0, 0.0, 0.0, -1.0, 0.0],
   [5.0, -1.0, 0.0, 0.0, 0.0],
   [5.0, 0.0, 1.0, 0.0, 7.05],
   [5.0, 0.0, 0.0, -1.0, 0.0],
   [6.0, -1.0, 0.0, 0.0, 0.0],
   [6.0, 0.0, 1.0, 0.0, -2.35],
   [6.0, 0.0, 0.0, -1.0, 0.0]]},
 3: {'chains': ['A', 'B'],
  'matrix': [[1.0, 1.0, 0.0, 0.0, 0.0],
   [1.0, 0.0, 1.0, 0.0, 0.0],
   [1.0, 0.0, 0.0, 1.0, 0.0]]}}

In [ ]:
parse_transformation(test.data_mmCIF)

_pdbx_struct_assembly_gen OrderedDict([('assembly_id', '1'), ('oper_expression', '1,2'), ('asym_id_list', 'A,B')])
_pdbx_struct_oper_list {'col_names': ['id', 'type', 'name', 'symmetry_operation', 'matrix[1][1]', 'matrix[1][2]', 'matrix[1][3]', 'vector[1]', 'matrix[2][1]', 'matrix[2][2]', 'matrix[2][3]', 'vector[2]', 'matrix[3][1]', 'matrix[3][2]', 'matrix[3][3]', 'vector[3]'], 'value': [['1', '2'], ["'identity operation'", "'crystal symmetry operation'"], ['1_555', '1_565'], ['x,y,z', 'x,y+1,z'], ['1.0000000000', '1.0000000000'], ['0.0000000000', '0.0000000000'], ['0.0000000000', '0.0000000000'], ['0.0000000000', '0.9401502764'], ['0.0000000000', '0.0000000000'], ['1.0000000000', '1.0000000000'], ['0.0000000000', '0.0000000000'], ['0.0000000000', '4.7875628934'], ['0.0000000000', '0.0000000000'], ['0.0000000000', '0.0000000000'], ['1.0000000000', '1.0000000000'], ['0.0000000000', '0.0000000000']]}


{1: {'chains': ['A', 'B'],
  'matrix': [[1.0, 1.0, 0.0, 0.0, 0.0],
   [1.0, 0.0, 1.0, 0.0, 0.0],
   [1.0, 0.0, 0.0, 1.0, 0.0],
   [2.0, 1.0, 0.0, 0.0, 0.9401502764],
   [2.0, 0.0, 1.0, 0.0, 4.7875628934],
   [2.0, 0.0, 0.0, 1.0, 0.0]]}}

In [ ]:

assert test.len == 56
# Here there is only identity matrix
# as symmetry, so no change is expected
test.add_symmetry()
assert test.len == 56
test.apply_transformation()
assert test.len == 112
test.copy_box(3, 3, 3)
assert test.len == 3024
test.compute_chains_CA()
test.remove_overlap_chain()
assert test.len == 3024

test.write("test.pdb", check_file_out=False)


In [ ]:
test.write('tmp.pdb', check_file_out=False)

In [ ]:
test_2 = Coor('tmp.pdb')
test_2.write('tmp2.gro', check_file_out=False)

In [ ]:
test.add_symmetry()
test.apply_transformation(index=1)
test.compute_chains_CA()
test.write_pdb('tmp.pdb', check_file_out=False)

Add symmetry 1
0
1
Add transformation 1
2
Add transformation 2


In [ ]:
test.symmetry

{'matrix': [[1.0, 1.0, 0.0, 0.0, 0.0],
  [1.0, 0.0, 1.0, 0.0, 0.0],
  [1.0, 0.0, 0.0, 1.0, 0.0],
  [2.0, -1.0, 0.0, 0.0, 0.0],
  [2.0, 0.0, 1.0, 0.0, 2.35],
  [2.0, 0.0, 0.0, -1.0, 0.0]]}

In [ ]:
test.models

In [ ]:
print(test.transformation)

{1: {'chains': ['A'], 'matrix': [[1.0, 1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 1.0, 0.0], [2.0, 1.0, 0.0, 0.0, 0.0], [2.0, 0.0, 1.0, 0.0, 4.7], [2.0, 0.0, 0.0, 1.0, 0.0], [3.0, 1.0, 0.0, 0.0, 0.0], [3.0, 0.0, 1.0, 0.0, -4.7], [3.0, 0.0, 0.0, 1.0, 0.0]]}, 2: {'chains': ['A'], 'matrix': [[1.0, -1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 2.35], [1.0, 0.0, 0.0, -1.0, 0.0], [2.0, -1.0, 0.0, 0.0, 0.0], [2.0, 0.0, 1.0, 0.0, 7.05], [2.0, 0.0, 0.0, -1.0, 0.0], [3.0, -1.0, 0.0, 0.0, 0.0], [3.0, 0.0, 1.0, 0.0, -2.35], [3.0, 0.0, 0.0, -1.0, 0.0]]}, 3: {'chains': ['A'], 'matrix': [[1.0, 1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 1.0, 0.0]]}}


In [ ]:
def parse_transformation(text):

    transformation_dict = {}

    for line in text.split('\n'):
        if line[11:23] == "BIOMOLECULE:":
            biomol = int(line[24:])
            transformation_dict[biomol] = {'chains': [], 'matrix': []}
        elif line[34:41] == "CHAINS:":
            transformation_dict[biomol]['chains'] += line[42:].split()
        elif line.startswith('REMARK 350   BIOMT'):
            transformation_dict[biomol]['matrix'] += [[float(x) for x in line[19:].split()]]
        #print(line, line[34:])
    print(transformation_dict)


parse_transformation(test.transformation)

{1: {'chains': ['A'], 'matrix': [[1.0, 1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 1.0, 0.0], [2.0, 1.0, 0.0, 0.0, 0.0], [2.0, 0.0, 1.0, 0.0, 4.7], [2.0, 0.0, 0.0, 1.0, 0.0], [3.0, 1.0, 0.0, 0.0, 0.0], [3.0, 0.0, 1.0, 0.0, -4.7], [3.0, 0.0, 0.0, 1.0, 0.0]]}, 2: {'chains': ['A'], 'matrix': [[1.0, -1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 2.35], [1.0, 0.0, 0.0, -1.0, 0.0], [2.0, -1.0, 0.0, 0.0, 0.0], [2.0, 0.0, 1.0, 0.0, 7.05], [2.0, 0.0, 0.0, -1.0, 0.0], [3.0, -1.0, 0.0, 0.0, 0.0], [3.0, 0.0, 1.0, 0.0, -2.35], [3.0, 0.0, 0.0, -1.0, 0.0]]}, 3: {'chains': ['A'], 'matrix': [[1.0, 1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 1.0, 0.0]]}}


In [ ]:
test.models.append(deepcopy(test.models[0]))
test.models.append(deepcopy(test.models[0]))

In [ ]:
text = \
"""REMARK 350   BIOMT1   2 -1.000000  0.000000  0.000000        0.00000            
REMARK 350   BIOMT2   2  0.000000  1.000000  0.000000        7.05000            
REMARK 350   BIOMT3   2  0.000000  0.000000 -1.000000        0.00000"""

rot_mat = []

for line in text.split("\n"):
    rot_mat.append([float(x) for x in line.split()[4:]])

rot_mat = np.array(rot_mat, dtype=object)
print(rot_mat)

text = \
"""REMARK 350   BIOMT1   3 -1.000000  0.000000  0.000000        0.00000            
REMARK 350   BIOMT2   3  0.000000  1.000000  0.000000       -2.35000            
REMARK 350   BIOMT3   3  0.000000  0.000000 -1.000000        0.00000"""

rot_mat_2 = []

for line in text.split("\n"):
    rot_mat_2.append([float(x) for x in line.split()[4:]])

rot_mat_2 = np.array(rot_mat_2, dtype=object)
print(rot_mat_2)

[[-1.0 0.0 0.0 0.0]
 [0.0 1.0 0.0 7.05]
 [0.0 0.0 -1.0 0.0]]
[[-1.0 0.0 0.0 0.0]
 [0.0 1.0 0.0 -2.35]
 [0.0 0.0 -1.0 0.0]]


In [ ]:
rot_mat[:,:3]

array([[-1.0, 0.0, 0.0],
       [0.0, 1.0, 0.0],
       [0.0, 0.0, -1.0]], dtype=object)

In [ ]:
test.models[1].xyz = np.dot(test.models[0].xyz, rot_mat[:,:3]) + rot_mat[:,3]
test.models[2].xyz = np.dot(test.models[0].xyz, rot_mat_2[:,:3]) + rot_mat_2[:,3]

In [ ]:
test.merge_model()
test.write_pdb('tmp.pdb', check_file_out=False)

In [ ]:
pwd

'/home/murail/Documents/Code/pdb_numpy'